# Task 2 Odd to Even

Design a quantum algorithm that when given numbers of range `[1,n)` and are odd convert them into even numbers, and they must stay in the same range so they cannot be less than 1 nor greater than n. `n = 2^k` where k is the number of qubits you are going to use.

Example:

`B = odd_to_even (n = 31,list= [1,2,2,4,5,6,7,11,17,21,22,23] )`

`print(B)`

One possible output is
 
`“[2,2,2,4,4,6,8,10,18,20,22,22]”`

Exist multiple solutions

References:

[1] Deutsch, David, and Richard Jozsa. "Rapid solution of problems by quantum computation." Proceedings of the Royal Society of London. Series A: Mathematical and Physical Sciences 439.1907 (1992): 553-558.

[2] Bernstein, Ethan, and Umesh Vazirani. "Quantum complexity theory." SIAM Journal on computing 26.5 (1997): 1411-1473.

[3] Grover, Lov K. , "A fast quantum mechanical algorithm for database search", Proceedings of the 28th Annual ACM Symposium on the Theory of Computing (1996), arXiv:quant-ph/9605043


#### Installing Modules

In [ ]:
%%bash
pip install qiskit
pip install qiskit_ibm_runtime
pip install qiskit[visualization]

#### Importing Functions

In [110]:
import math
from qiskit.circuit.library import GroverOperator, MCMT, ZGate
from qiskit import transpile, QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.providers.basic_provider import BasicProvider

#### Choosing Backend Provider

In [111]:
backend = BasicProvider().get_backend('basic_simulator')

#### Grover's Algorithm Oracle

In [112]:
# In Grover's Algorithm the "oracle" marks the states corresponding to even numbers
# The 0th qubit is always in state |0> because the bit-string of even numbers always ends with 0

def groveroracle(markedstates, num_qubits):
    """Build Grover's Algorithm oracle for multiple marked states

    Parameters:
        markedstates (list): Marked states of oracle
        num_qubits (int): Number of qubits

    Returns:
        QuantumCircuit: Quantum circuit representing Grover's Algorithm oracle
    """
    qc = QuantumCircuit(num_qubits)

    # keep array of marked even numbers to avoid marking twice
    markedevennums = []

    # Mark each target state in the input list
    for target in markedstates:
        # Flip target bit-string to match Qiskit bit-ordering
        flippedtarget = target[::-1]
        if flippedtarget[1:] not in markedevennums:
            # Find indices of all the '0' elements in bit-string
            zeroindices = [0] # 0th qubit is always 0 for even numbers
            zeroindices.extend([ind for ind in range(1, num_qubits) if flippedtarget.startswith("0", ind)])
            # Add a multi-controlled Z-gate with pre- and post-applied X-gates (open-controls)
            # where the target bit-string has a '0' entry
            qc.x(zeroindices)
            qc.compose(MCMT(ZGate(), num_qubits - 1, 1), inplace=True)
            qc.x(zeroindices)
            markedevennums.append(flippedtarget[1:])
    return qc
     

#### Odd to Even

In [113]:
def oddtoeven(n, convertedlist):
    """
    Build a function that uses quantum circuit to convert
    all odd numbers to even numbers

    Parameters:
        n (int): The maximum number
        convertedlist (list): List of numbers

    Returns:
        list: A list where all numbers are even
    """
    # Calculate the number of required qubits
    k = math.ceil(math.log2(n))

    # Convert the list of numbers from decimal to binary
    markedstates = [bin(num)[2:].zfill(k) for num in convertedlist]

    # Prepare an oracle for the marked states
    oracle = groveroracle(markedstates, k)

    # Count the number of multi-control z gate in the oracle
    # This represents the number of unique marked elements
    uniquemarkednums = dict(oracle.count_ops())["ccx" if k==3 else f"c{k-1}z"]

    # Append the diffusion operator
    groverdiffop = GroverOperator(oracle)

    # Determine the optimal number of iterations
    optimalnumofiterations = max(1, math.floor(
        math.pi / (4 * math.asin(math.sqrt(5 / 2**groverdiffop.num_qubits)))
        ))

    # Prepare the Quantum Circuit
    qc = QuantumCircuit(k)

    # Create even superposition of all basis states
    qc.h(range(groverdiffop.num_qubits))

    # Apply Grover operator the optimal number of times
    qc.compose(groverdiffop.power(optimalnumofiterations), inplace=True)

    # Measure all qubits
    qc.measure_all()

    # Draw the final circuit
    qc.decompose().draw(output="mpl", style="iqp")

    # Simulate the circuit
    tqc = transpile(qc, backend)
    counts = backend.run(tqc, shots=10000).result().get_counts()

    # Selected even numbers
    markedeven = sorted(counts, key=counts.get, reverse=True)[:uniquemarkednums]

    # Convert to decimal
    markedeven = [int(j, 2) for j in markedeven]
    # Delete 0 from the list since we have to remain within [1,n)
    if 0 in markedeven:
        markedeven.remove(0)

    # Prepare the even list
    evenlist = []
    for j in convertedlist:
        if j in markedeven:
            evenlist.append(j)
        elif j+1 in markedeven:
            evenlist.append(j+1)
        else:
            evenlist.append(j-1)

    return evenlist

In [117]:
n = 31
convertedlist = [1,2,2,4,5,6,7,11,17,21,22,23]
evenlist = oddtoeven(n, convertedlist)
print(evenlist)

[2, 2, 2, 4, 6, 6, 6, 10, 16, 22, 22, 22]
